# SQL in Pandas with SQLAlchemy

We will use the "sshtunnel" library to connect to our remote AWS instance and then pull some data into Pandas using SQLAlchemy and Pyscopg2.

## Creating the SSH Tunnel

The "sshtunnel" library can read an SSH config file, so creating a tunnel is quite easy assuming SSH keys are setup and the SSH config entry has been created. With this setup, the "sshtunnel" library automatically determines what the address of the local port should be.

In [3]:
from sshtunnel import SSHTunnelForwarder

AWS_IP_ADDRESS = '34.217.58.114'
AWS_USERNAME = 'anjali'
SSH_KEY_PATH = '/Users/user/.ssh/id_rsa'

server = SSHTunnelForwarder(
    AWS_IP_ADDRESS,
    ssh_username=AWS_USERNAME,
    ssh_pkey=SSH_KEY_PATH,
    remote_bind_address=('localhost', 5432),
)

server.start()
print(server.is_active, server.is_alive, server.local_bind_port)

True True 49999


##  Connecting via Python
We'll be using a Psycopg2 connector alongside SQLAlchemy to connect to this database.

* **SQLAlchemy:** generates SQL statements
* **Psycopg2:** sends the SQL statements to the Postgres database

    Let's make the connection to the database. Note that the IP address of the Postgres database is 'localhost' and the port is set to whatever the `server` connection above contains. This is because we have used the SSH tunnel to create a connection between the AWS instance and our computer. SSH tunnels enable remote instances to behave as if they are *local*.

In [4]:
from sqlalchemy import create_engine

# Postgres username, password, and database name
POSTGRES_IP_ADDRESS = 'localhost' ## This is localhost because SSH tunnel is active
POSTGRES_PORT = str(server.local_bind_port)
POSTGRES_USERNAME = 'anjali'     ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = 'Brandywine10' ## CHANGE THIS TO YOUR POSTGRES PASSWORD
POSTGRES_DBNAME = 'baseball'

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME, 
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_IP_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

# Create the connection
cnx = create_engine(postgres_str)

## Load Some Data!

Pandas has a `read_sql_query` method that will pass a SQL statement to a database connection. Here is an example from the all-star table.

In [12]:
import pandas as pd
df = pd.read_csv('2013_movies.csv')
df.head()

,Title,Budget,DomesticTotalGross,Director,Rating,Runtime,ReleaseDate
0,The Hunger Games: Catching Fire,130000000.0,424668047,Francis Lawrence,PG-13,146,2013-11-22 00:00:00
1,Iron Man 3,200000000.0,409013994,Shane Black,PG-13,129,2013-05-03 00:00:00
2,Frozen,150000000.0,400738009,Chris BuckJennifer Lee,PG,108,2013-11-22 00:00:00
3,Despicable Me 2,76000000.0,368061265,Pierre CoffinChris Renaud,PG,98,2013-07-03 00:00:00
4,Man of Steel,225000000.0,291045518,Zack Snyder,PG-13,143,2013-06-14 00:00:00


In [13]:
df.to_sql('movies', cnx, index=False)

In [14]:
pd.read_sql_query('''SELECT * FROM movies LIMIT 5;''', cnx)

,Title,Budget,DomesticTotalGross,Director,Rating,Runtime,ReleaseDate
0,The Hunger Games: Catching Fire,130000000.0,424668047,Francis Lawrence,PG-13,146,2013-11-22 00:00:00
1,Iron Man 3,200000000.0,409013994,Shane Black,PG-13,129,2013-05-03 00:00:00
2,Frozen,150000000.0,400738009,Chris BuckJennifer Lee,PG,108,2013-11-22 00:00:00
3,Despicable Me 2,76000000.0,368061265,Pierre CoffinChris Renaud,PG,98,2013-07-03 00:00:00
4,Man of Steel,225000000.0,291045518,Zack Snyder,PG-13,143,2013-06-14 00:00:00


In [65]:


pd.read_sql_query('''SELECT * FROM allstarfull LIMIT 5;''', cnx)

,playerid,yearid,gamenum,gameid,teamid,lgid,gp,startingpos
0,gomezle01,1933,0,ALS193307060,NYA,AL,1,1
1,ferreri01,1933,0,ALS193307060,BOS,AL,1,2
2,gehrilo01,1933,0,ALS193307060,NYA,AL,1,3
3,gehrich01,1933,0,ALS193307060,DET,AL,1,4
4,dykesji01,1933,0,ALS193307060,CHA,AL,1,5


And another from the schools table.

In [66]:
pd.read_sql_query('''SELECT * FROM schools LIMIT 5;''', cnx)

,schoolid,schoolname,schoolcity,schoolstate,schoolnick
0,abilchrist,Abilene Christian University,Abilene,TX,Wildcats
1,adelphi,Adelphi University,Garden City,NY,Panthers
2,adrianmi,Adrian College,Adrian,MI,Bulldogs
3,airforce,United States Air Force Academy,Colorado Springs,CO,Falcons
4,akron,University of Akron,Akron,OH,Zips


More sophisticated queries can also be used. This example finds the states with the most schools.

In [67]:
pd.read_sql_query('''SELECT * FROM salaries LIMIT 5;''', cnx)

,yearid,teamid,lgid,playerid,salary
0,1985,BAL,AL,murraed02,1472819.0
1,1985,BAL,AL,lynnfr01,1090000.0
2,1985,BAL,AL,ripkeca01,800000.0
3,1985,BAL,AL,lacyle01,725000.0
4,1985,BAL,AL,flanami01,641667.0


In [68]:
pd.read_sql_query('''SELECT * FROM franchises LIMIT 5;''', cnx)

,franchid,franchname,active,nassoc
0,ALT,Altoona Mountain City,N,None
1,ANA,Los Angeles Angels of Anaheim,Y,None
2,ARI,Arizona Diamondbacks,Y,None
3,ATH,Philadelphia Athletics,N,PNA
4,ATL,Atlanta Braves,Y,BNA


In [69]:
# question 1
sql_query = '''SELECT franchises.franchid,franchises.franchname,salaries.yearid, SUM(salaries.salary) AS total_spent
               FROM salaries
               JOIN franchises ON salaries.teamid = franchises.franchid
               GROUP BY  franchises.franchid, salaries.teamid, salaries.yearid 
               ORDER BY franchises.franchid, salaries.yearid 
               LIMIT 5;'''

pd.read_sql_query(sql_query, cnx)

,franchid,franchname,yearid,total_spent
0,ANA,Los Angeles Angels of Anaheim,1997,31135472.0
1,ANA,Los Angeles Angels of Anaheim,1998,41281000.0
2,ANA,Los Angeles Angels of Anaheim,1999,55388166.0
3,ANA,Los Angeles Angels of Anaheim,2000,51464167.0
4,ANA,Los Angeles Angels of Anaheim,2001,47535167.0


In [70]:
pd.read_sql_query('''SELECT * FROM fielding LIMIT 5;''', cnx)

,playerid,yearid,stint,teamid,lgid,pos,g,gs,innouts,po,a,e,dp,pb,wp,sb,cs,zr
0,ansonca01,1871,1,RC1,NA,1B,1,None,None,7,0,0,0,None,None,None,None,None
1,biermch01,1871,1,FW1,NA,1B,1,None,None,9,0,2,0,None,None,None,None,None
2,carleji01,1871,1,CL1,NA,1B,29,None,None,295,4,34,10,None,None,None,None,None
3,connone01,1871,1,TRO,NA,1B,4,None,None,35,1,5,1,None,None,None,None,None
4,cravebi01,1871,1,TRO,NA,1B,2,None,None,18,1,1,1,None,None,None,None,None


In [71]:
# question 2
sql_query = '''SELECT playerid, MIN(yearid) AS first_year, MAX(yearid) AS last_year
               FROM fielding
               GROUP BY  playerid
               LIMIT 5;'''

pd.read_sql_query(sql_query, cnx)

,playerid,first_year,last_year
0,aardsda01,2004,2013
1,aaronha01,1954,1976
2,aaronto01,1962,1971
3,aasedo01,1977,1990
4,abadan01,2001,2003


In [72]:
# question 3
sql_query = '''SELECT playerid, COUNT(gameid) AS all_star_games_played
               FROM allstarfull
               GROUP BY  playerid
               ORDER BY all_star_games_played DESC
               LIMIT 5;'''

pd.read_sql_query(sql_query, cnx)

,playerid,all_star_games_played
0,aaronha01,25
1,mayswi01,24
2,musiast01,24
3,mantlmi01,20
4,willite01,19


In [124]:
pd.read_sql_query('''SELECT * FROM schoolsplayers LIMIT 5;''', cnx)

,playerid,schoolid,yearmin,yearmax
0,aardsda01,pennst,2001,2001
1,aardsda01,rice,2002,2003
2,abbeybe01,vermont,1888,1892
3,abbotgl01,carkansas,1970,1970
4,abbotje01,kentucky,1991,1992


In [127]:
# question 4
sql_query = '''SELECT schoolid, COUNT (DISTINCT playerid) AS num_players
               FROM schoolsplayers
               GROUP BY  schoolid
               ORDER BY num_players DESC
               LIMIT 5;'''

pd.read_sql_query(sql_query, cnx)

,schoolid,num_players
0,usc,102
1,texas,100
2,arizonast,98
3,stanford,82
4,michigan,77


In [105]:
pd.read_sql_query('''SELECT * FROM mast LIMIT 5;''', cnx)

,playerid,birthyear,birthmonth,birthday,birthcountry,birthstate,birthcity,deathyear,deathmonth,deathday,...,namelast,namegiven,weights,height,bats,throws,debut,finalgame,retroid,bbrefid
0,aardsda01,1981,12,27,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,205,75,R,R,2004-04-06,2013-09-28,aardd001,aardsda01
1,aaronha01,1934,2,5,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180,72,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939,8,5,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190,75,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954,9,8,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190,75,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972,8,25,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184,73,L,L,2001-09-10,2006-04-13,abada001,abadan01


In [94]:
# question 5
sql_query = '''SELECT playerid, debut, finalgame, finalgame - debut AS career_duration_in_days
               FROM mast
               WHERE debut IS NOT NULL AND finalgame is NOT NULL 
               ORDER BY career_duration_in_days DESC
               LIMIT 5;'''

pd.read_sql_query(sql_query, cnx)

,playerid,debut,finalgame,career_duration_in_days
0,altroni01,1898-07-14,1933-10-01,12862
1,orourji01,1872-04-26,1904-09-22,11836
2,minosmi01,1949-04-19,1980-10-05,11492
3,olearch01,1904-04-14,1934-09-30,11126
4,lathaar01,1880-07-05,1909-09-30,10678


In [104]:
# question 6
sql_query = '''SELECT EXTRACT(MONTH FROM debut ) AS month, COUNT(playerid) AS number_of_debuts    
               FROM mast 
               WHERE debut IS NOT NULL
               GROUP BY month
               ORDER BY month
               LIMIT 20;'''

pd.read_sql_query(sql_query, cnx)

,month,number_of_debuts
0,3.0,41
1,4.0,4711
2,5.0,2230
3,6.0,1893
4,7.0,1978
5,8.0,1943
6,9.0,5061
7,10.0,308


In [118]:
# question 7 - salaries on master
sql_query = '''SELECT salaries.playerid, AVG(salary) AS avg_salary
               FROM salaries
               LEFT JOIN mast ON salaries.playerid = mast.playerid
               GROUP BY salaries.playerid
               ORDER BY salaries.playerid
               LIMIT 5;'''
pd.read_sql_query(sql_query, cnx)

,playerid,avg_salary
0,aardsda01,1.322821e+06
1,aasedo01,5.750000e+05
2,abadan01,3.270000e+05
3,abadfe01,4.515000e+05
4,abbotje01,2.462500e+05


In [121]:
# question 7 - master on salaries
sql_query = '''SELECT mast.playerid, AVG(salary) AS avg_salary
               FROM mast
               LEFT JOIN salaries ON mast.playerid = salaries.playerid
               GROUP BY mast.playerid
               ORDER BY mast.playerid
               LIMIT 5;'''
pd.read_sql_query(sql_query, cnx)

,playerid,avg_salary
0,aardsda01,1.322821e+06
1,aaronha01,NaN
2,aaronto01,NaN
3,aasedo01,5.750000e+05
4,abadan01,3.270000e+05


2018-10-18 17:11:51,057| ERROR   | Socket exception: Connection reset by peer (54)


Using left join  it matters whether we join master on salary or the other way round. As we can see if we join master on salary ther are many NaN values because every playerid who is mentioned in the master may not be on the salaries data. 

In [13]:
sql_query = '''SELECT schoolstate as state, Count(schoolid) as ct 
               FROM schools 
               GROUP BY state 
               ORDER BY ct DESC 
               LIMIT 5;'''

pd.read_sql_query(sql_query, cnx)

,state,ct
0,PA,57
1,CA,48
2,NY,45
3,TX,41
4,OH,33


Finally, this example finds five players from the year 1985 whose salary was above $500,000.

In [14]:
sql_query = '''SELECT playerid, salary 
               FROM Salaries 
               WHERE yearid = '1985' AND salary > '500000'
               ORDER BY salary DESC
               LIMIT 5;'''

pd.read_sql_query(sql_query, cnx)

,playerid,salary
0,schmimi01,2130300.0
1,cartega01,2028571.0
2,fostege01,1942857.0
3,winfida01,1795704.0
4,gossari01,1713333.0


## Close Server Connection

Finally, we should close the server connection when complete.

In [15]:
server.close()